<a href="https://colab.research.google.com/github/mayapatri/mayapatri.github.io/blob/main/Lyme_Incident_Rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Expansion of Lyme Cases in the United States Over 25 years


---


### Maya Patri and Dylan Murray

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from google.colab import drive
drive.mount('/content/drive')

%cd /content
!git clone https://github.com/mayapatri/mayapatri.github.io/blob/main/data_science_project.ipynb
%cd /content/drive/

#df = pd.read_csv('/content/drive/My Drive/DataScienceProject/LymeDisease_9211_county.csv')
df = pd.read_csv('../LymeDisease_9211_county.csv')

df.rename(columns={'ConfirmedCount_1992_1996': 'Lyme Cases 1992-1996'}, inplace=True)
df.rename(columns={'ConfirmedCount_1997_2001': 'Lyme Cases 1997-2001'}, inplace=True)
df.rename(columns={'ConfirmedCount_2002_2006': 'Lyme Cases 2002-2006'}, inplace=True)
df.rename(columns={'ConfirmedCount_2007_2011': 'Lyme Cases 2007-2011'}, inplace=True)

df.fillna(0, inplace=True)

df['StateCode'] = df['StateCode'].astype(int)
df['CountyCode'] = df['CountyCode'].astype(int)

df['StateName'] = df['StateName'].astype(str)
df['CountyName'] = df['CountyName'].astype(str)

df['Lyme Cases 1992-1996'] = df['Lyme Cases 1992-1996'].astype(int)
df['Lyme Cases 1997-2001'] = df['Lyme Cases 1997-2001'].astype(int)
df['Lyme Cases 2002-2006'] = df['Lyme Cases 2002-2006'].astype(int)
df['Lyme Cases 2007-2011'] = df['Lyme Cases 2007-2011'].astype(int)

df_long = df.melt(
    id_vars=['StateName', 'CountyName', 'StateCode', 'CountyCode'],
    value_vars=[
        'Lyme Cases 1992-1996',
        'Lyme Cases 1997-2001',
        'Lyme Cases 2002-2006',
        'Lyme Cases 2007-2011'
    ],
    var_name='YearRange',
    value_name='Cases'
)

# reshaped data
print(df_long.head())

# aggregate national cases per year range
national_cases = df_long.groupby('YearRange')['Cases'].sum().reset_index()

# plot national cases over the time periods
plt.figure(figsize=(10,6))
sns.barplot(data=national_cases, x='YearRange', y='Cases', palette='Blues')

plt.title('Total Lyme Disease Cases in the U.S. by Year Range')
plt.xlabel('Year Range')
plt.ylabel('Total Cases')
plt.grid(axis='y')
plt.show()

# aggregate cases by state and sort states by total cases
state_cases = df_long.groupby('StateName')['Cases'].sum().reset_index().sort_values(by='Cases', ascending=False)

# get top 5 states
top_states = state_cases.head(5)['StateName'].tolist()

# filter the dataset for top 5 states
top_states_df = df_long[df_long['StateName'].isin(top_states)]

# plotting
plt.figure(figsize=(12,8))
sns.lineplot(data=top_states_df, x='YearRange', y='Cases', hue='StateName', marker='o')

plt.title('Lyme Disease Trends (Top 5 States)')
plt.xlabel('Year Range')
plt.ylabel('Number of Cases')
plt.legend(title='State')
plt.grid(True)
plt.show()

# county-level visualization for a specific state NewYork
ny_df = df_long[df_long['StateName'] == 'New York']

# aggregate county cases
county_cases_ny = ny_df.groupby('CountyName')['Cases'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12,8))
county_cases_ny.plot(kind='bar', color='skyblue')
plt.title('Top 10 Counties in New York by Lyme Disease Cases (1992-2011)')
plt.xlabel('County')
plt.ylabel('Number of Cases')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
Cloning into 'data_science_project.ipynb'...
fatal: repository 'https://github.com/mayapatri/mayapatri.github.io/blob/main/data_science_project.ipynb/' not found
/content/drive


FileNotFoundError: [Errno 2] No such file or directory: '../LymeDisease_9211_county.csv'